In [14]:
import csv
import os
path = 'Dissertation/Test'
with open('images2.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    i=0
    for row in csvReader:
        if i==0:
            i=2
            continue
        filename=row[0]
        filepath=os.path.join(path,filename)
        all_files=os.listdir(filepath)
        for x in all_files:
            newfilename=x.split(".")[0]+"_"+row[1]+"_"+row[2]+"."+x.split(".")[1]
            newfilename_path=os.path.join(filepath,newfilename)
            oldfilename_path=os.path.join(filepath,x)
            os.rename(oldfilename_path,newfilename_path)

In [6]:
import os
path = 'Dissertation/Test'
files = os.listdir(path)
for file in files:
    print(file)
    
   #os.rename(os.path.join(path, file), os.path.join(path, 'xyz_' + file + '.csv'))

MBA07
.DS_Store
EAE01
EEJ02
MBA04
MBA03
MBA02
EAE02
EAE03
EAE04
EEJ06
EEJ09
